# Test the three models

In [1]:
import keras
from keras import backend as K
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate, Add, Lambda
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D, MaxPooling2D
from keras.models import Sequential, Model
from keras.optimizers import Adam, RMSprop

import datetime
import matplotlib.pyplot as plt
import numpy as np
import scipy
import sys
import os

from data_helper import predict_15k, save_hist, save_model
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, History

from time import gmtime, strftime
from data_helper import readImg, readImgInv, imagePatches, removeBlackImg, removeCorrespondence, check_and_create

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
smooth = 1.

def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_coef_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)

def iou_coef(y_true, y_pred):
    dice = dice_coef(y_true, y_pred)
    iou = dice / (2 - dice)
    return iou

def iou_loss(y_true, y_pred):
    return 1 - iou_coef(y_true, y_pred)

In [3]:
# Residual u-net 
# Reference: https://github.com/DuFanXin/deep_residual_unet/blob/master/res_unet.py

def res_block(x, nb_filters, strides, increase = False):
    # This implementation used the double 3x3 structure and followed the Identity Mappings
    res_path = BatchNormalization()(x)
    res_path = Activation(activation='relu')(res_path)
    
    res_path = Conv2D(filters=nb_filters[0], kernel_size=(3, 3), padding='same', strides=strides[0])(res_path)
    res_path = BatchNormalization()(res_path)
    res_path = Activation(activation='relu')(res_path)
    res_path = Conv2D(filters=nb_filters[1], kernel_size=(3, 3), padding='same', strides=strides[1])(res_path)
    
    if increase:
        shortcut = Conv2D(nb_filters[1], kernel_size=(1, 1), strides=strides[0])(x)
        shortcut = BatchNormalization()(shortcut)
    else:
        shortcut = x

    res_path = Add()([shortcut, res_path])
    return res_path

def decoder(x, from_encoder):
    main_path = UpSampling2D(size=(2, 2))(x)
    main_path = Concatenate(axis=3)([main_path, from_encoder[2]])
    main_path = res_block(main_path, [128, 128], [(1, 1), (1, 1)], increase = True)

    main_path = UpSampling2D(size=(2, 2))(main_path) 
    main_path = Concatenate(axis=3)([main_path, from_encoder[1]])
    main_path = res_block(main_path, [64, 64], [(1, 1), (1, 1)], increase = True)

    main_path = UpSampling2D(size=(2, 2))(main_path)
    main_path = Concatenate(axis=3)([main_path, from_encoder[0]])
    main_path = res_block(main_path, [32, 32], [(1, 1), (1, 1)], increase = True)

    return main_path

def encoder(x):
    to_decoder = []

    main_path = Conv2D(filters=32, kernel_size=(3, 3), padding='same', strides=(1, 1))(x)
    main_path = BatchNormalization()(main_path)
    main_path = Activation(activation='relu')(main_path)
    main_path = Conv2D(filters=32, kernel_size=(3, 3), padding='same', strides=(1, 1))(main_path)

    shortcut = Conv2D(filters=32, kernel_size=(1, 1), strides=(1, 1))(x)
    shortcut = BatchNormalization()(shortcut)

    main_path = Add()([shortcut, main_path])
    # first branching to decoder
    to_decoder.append(main_path)

    main_path = res_block(main_path, [64, 64], [(2, 2), (1, 1)], increase = True)
    to_decoder.append(main_path)

    main_path = res_block(main_path, [128, 128], [(2, 2), (1, 1)], increase = True)
    to_decoder.append(main_path)

    return to_decoder

In [4]:
class GAN4MapGen(): # Based on u-net, residual u-net and pix2pix
    # Reference: https://github.com/eriklindernoren/Keras-GAN/blob/master/pix2pix/pix2pix.py
    
    def __init__(self):

        # Input shape
        self.img_rows = 128
        self.img_cols = 128
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        
        self.clip_value = 0.01
        
        # Configure data loader
        self.dataset_name = 'mapgen'

        # Calculate output shape of D (PatchGAN)
        patch = int(self.img_rows / 2**4)
        self.disc_patch = (patch, patch, 1)
        
        # Calculate output shape of D (PatchGAN) better version
        self.patch_size = 32
        self.nb_patches = int((self.img_rows / self.patch_size) * (self.img_cols / self.patch_size))
        self.patch_gan_dim = (self.patch_size, self.patch_size, self.channels)
        
        # Number of filters in the first layer of G and D
        self.gf = 64
        self.df = 64

        #optimizer = Adam(0.0002, 0.5) # Original
        optimizer = Adam(0.0001, 0.5) # Original # Latest achieved by 0.00008
        #optimizer = Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08) # An old version of Pix2pix

        #-------------------------
        # Construct Computational
        #   Graph of Generator
        #-------------------------

        # Build the generator
        #self.generator = self.build_generator() # Old generator from 
        self.generator = self.build_res_unet_generator()

        # Input images and their conditioning images
        #img_A = Input(shape=self.img_shape) # Target
        img_B = Input(shape=self.img_shape) # Input

        # By conditioning on B generate a fake version of A
        fake_A = self.generator(img_B)
        
        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])
        
        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # Discriminators determines validity of translated images / condition pairs
        #valid = self.discriminator([fake_A, img_B])
        #self.combined = Model(inputs=[img_A, img_B], outputs=[valid, fake_A])
        
        valid = self.discriminator([fake_A])
        self.combined = Model(inputs= img_B, outputs=[valid, fake_A])
        
        # Original Pix2Pix - low weight for discriminator
        self.combined.compile(loss=['mse', 'mae'],
                              loss_weights=[1, 100], # [1, 100] is okay
                              optimizer=optimizer, metrics=['accuracy'])

    
    def build_res_unet_generator(self):
        """Residual U-Net Generator"""
        
        inputs = Input(shape=self.img_shape)
        to_decoder = encoder(inputs)
        path = res_block(to_decoder[2], [256, 256], [(2, 2), (1, 1)], increase = True) # 3x
        path = res_block(path, [256, 256], [(1, 1), (1, 1)]) # Number of block of bottleneck = 1
        path = res_block(path, [256, 256], [(1, 1), (1, 1)]) # Try to add one 2019.01.14, achieved best result ever
        path = decoder(path, from_encoder=to_decoder)
        path = Conv2D(filters=1, kernel_size=(1, 1), activation='sigmoid')(path) 

        return Model(input=inputs, output=path)
        
    def build_generator(self):
        """U-Net Generator"""

        def conv2d(layer_input, filters, f_size=4, bn=True):
            """Layers used during downsampling"""
            d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
            d = LeakyReLU(alpha=0.2)(d)
            if bn:
                d = BatchNormalization(momentum=0.8)(d)
            return d

        def deconv2d(layer_input, skip_input, filters, f_size=4, dropout_rate=0):
            """Layers used during upsampling"""
            u = UpSampling2D(size=2)(layer_input)
            u = Conv2D(filters, kernel_size=f_size, strides=1, padding='same', activation='relu')(u)
            if dropout_rate:
                u = Dropout(dropout_rate)(u)
            u = BatchNormalization(momentum=0.8)(u)
            u = Concatenate()([u, skip_input])
            return u

        # Image input
        d0 = Input(shape=self.img_shape)

        # Downsampling
        d1 = conv2d(d0, self.gf, bn=False)
        d2 = conv2d(d1, self.gf*2)
        d3 = conv2d(d2, self.gf*4)
        d4 = conv2d(d3, self.gf*8)
        d5 = conv2d(d4, self.gf*8)
        d6 = conv2d(d5, self.gf*8)
        d7 = conv2d(d6, self.gf*8)

        # Upsampling
        u1 = deconv2d(d7, d6, self.gf*8)
        u2 = deconv2d(u1, d5, self.gf*8)
        u3 = deconv2d(u2, d4, self.gf*8)
        u4 = deconv2d(u3, d3, self.gf*4)
        u5 = deconv2d(u4, d2, self.gf*2)
        u6 = deconv2d(u5, d1, self.gf)

        u7 = UpSampling2D(size=2)(u6)
        output_img = Conv2D(self.channels, kernel_size=4, strides=1, padding='same', activation='tanh')(u7)

        return Model(d0, output_img)

    def build_discriminator(self):

        def d_layer(layer_input, filters, f_size=3, bn=True): # Chnaged here for the order of bn and activation
            """Discriminator layer"""
            d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
            if bn:
                d = BatchNormalization(momentum=0.8)(d)
            d = Activation(activation='relu')(d)
            #d = LeakyReLU(alpha=0.2)(d)
            return d

        img_A = Input(shape=self.img_shape)
        
        # !!!! This step was deleted because the condition do not help in this case
        #img_B = Input(shape=self.img_shape)
        ## Concatenate image and conditioning image by channels to produce input
        #combined_imgs = Concatenate(axis=-1)([img_A, img_B])
        #d1 = d_layer(combined_imgs, self.df, bn=False)
        
        d1 = d_layer(img_A, self.df, bn=False)
        d2 = d_layer(d1, self.df*2)
        d3 = d_layer(d2, self.df*4)
        d4 = d_layer(d3, self.df*8)

        validity = Conv2D(1, kernel_size=3, strides=1, padding='same')(d4)

        return Model([img_A], validity)
    
    def train_generator_only(self, x_train_sim, y_train_sim, x_test_sim, y_test_sim, outPath):
        
        start_time = datetime.datetime.now()
        
        gen = self.build_res_unet_generator()
        #optimizer = Adam(0.0001, 0.5) 
        #optimizer = Adam()
        optimizer = Adam(0.0004) # Current best
        #optimizer = RMSprop() #
        #optimizer = RMSprop(lr=0.0001) # Worse
        
        # Loss function finished
        gen.compile(loss='mse', optimizer=optimizer, metrics=['accuracy']) # mse better than mae
        #gen.compile(loss='mae', optimizer=optimizer, metrics=['accuracy'])
        #gen.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
        #gen.compile(loss=dice_coef_loss, optimizer=optimizer, metrics=['accuracy'])

        data_gen_args = dict(rotation_range=180.)
        image_datagen = ImageDataGenerator(**data_gen_args)
        mask_datagen = ImageDataGenerator(**data_gen_args)
        
        seed = 1
        BATCH_SIZE = 16
        result_generator = zip(image_datagen.flow(x_train_sim, batch_size=BATCH_SIZE, seed=seed), 
                               mask_datagen.flow(y_train_sim, batch_size=BATCH_SIZE, seed=seed))
        
        History1 = History()
        hist1 = gen.fit_generator( result_generator,
                                              epochs = 100,
                                              steps_per_epoch=2000,
                                              verbose=1,
                                              shuffle=True,
                                              callbacks=[History1, 
                                                         EarlyStopping(patience=3), 
                                                         ReduceLROnPlateau(patience = 3, verbose = 0),
                                                         ModelCheckpoint(outPath + "weights.h5", 
                                                                         save_best_only = True, 
                                                                         save_weights_only = False)],
                                              validation_data=(x_test_sim, y_test_sim))
        save_hist(History1, outPath)
        
    
    def train(self, x_train_sim, y_train_sim, x_test_sim, y_test_sim, outPath, epochs, batch_size=1, sample_interval=50):

        start_time = datetime.datetime.now()

        # Adversarial loss ground truths
        valid = np.ones((batch_size,) + self.disc_patch)
        fake = np.zeros((batch_size,) + self.disc_patch)
        
        total_samples = len(x_train_sim)
        ids = np.arange(total_samples)
        np.random.shuffle(ids)
        n_batches = int(total_samples / batch_size)
        
        for epoch in range(epochs):
            for batch_i, (imgs_A, imgs_B) in enumerate(load_batch(x_train_sim, y_train_sim, batch_size)):

                # ---------------------
                #  Train Discriminator
                # ---------------------

                # Condition on B and generate a translated version
                fake_A = self.generator.predict(imgs_B)
                
                # Train the discriminators (original images = real / generated = Fake)
                d_loss_real = self.discriminator.train_on_batch([imgs_A], valid)
                d_loss_fake = self.discriminator.train_on_batch([fake_A], fake)
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
                
                # Clip critic weights
                for l in self.discriminator.layers:
                    weights = l.get_weights()
                    weights = [np.clip(w, -self.clip_value, self.clip_value) for w in weights]
                    l.set_weights(weights)

                # -----------------
                #  Train Generator
                # -----------------

                # Train the generators
                self.discriminator.trainable = False
                g_loss = self.combined.train_on_batch(imgs_B, [valid, imgs_A])
                self.discriminator.trainable = True
                
                elapsed_time = datetime.datetime.now() - start_time

                # Plot the progress
                if batch_i % sample_interval == 0:
                    
                    valid_test = np.ones((len(x_test_sim),) + self.disc_patch)
                    t_loss = self.combined.evaluate(x_test_sim, [valid_test, y_test_sim], verbose=0)
                    
                    print ("[Epoch %d/%d-%d/%d] [D loss&acc: %.3f, %.3f%%] [G loss&accA&accB: %.3f, %.3f%%, %.3f%%] [Test loss&acc: %.3f, %.3f%%, %.3f%%] time: %s" % (epoch, epochs,
                                                                                batch_i, n_batches,
                                                                                d_loss[0], 100*d_loss[1],
                                                                                100*g_loss[2], 100*g_loss[3], 100*g_loss[4],
                                                                                100*t_loss[2], 100*t_loss[3], 100*t_loss[4],
                                                                                elapsed_time))                 
            if epoch > 0:
                self.generator.save(outPath + 'model_epoch'+ str(epoch) +'.h5')

In [5]:
# Order the image dimension acc. to TensorFlow (batc_hsize, rows, cols, channels)
K.set_image_dim_ordering('tf')

scale = 15
p_size_1 = 128 # Compared with 256, which larger may generate round corners
trainPath = r"../tmp_data/data_feng/geb" + str(scale) +  "/"

# save image patch arrays
x_train_sim = np.load(trainPath + "x_train_sim.npy")
y_train_sim = np.load(trainPath + "y_train_sim.npy")
x_test_sim = np.load(trainPath + "x_test_sim.npy")
y_test_sim = np.load(trainPath + "y_test_sim.npy")

input_shape1 = (None, None, 1) #x_train_sim[0].shape
print('Input Shape of the trains', x_train_sim.shape)
print('Input Shape of the tests', x_test_sim.shape)

Input Shape of the trains (32289, 128, 128, 1)
Input Shape of the tests (3587, 128, 128, 1)


### Train residual unet

In [ ]:
############ Path Setting ##############
outPath = r"../tmp_results/predictions/"
timestr = strftime("%Y-%m-%d %H-%M-%S", gmtime())
outPath = outPath + timestr + '_' + str(scale)+ "/"
check_and_create(outPath)
print(outPath)

../tmp_results/predictions/2019-02-27 15-26-11_15/


In [ ]:
gan = GAN4MapGen()
gan.train_generator_only(x_train_sim, y_train_sim, x_test_sim, y_test_sim, outPath)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:81: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`


Epoch 1/100
2000/2000 [==============================] - 249s 125ms/step - loss: 0.0077 - acc: 0.9671 - val_loss: 0.0083 - val_acc: 0.9902
Epoch 2/100
2000/2000 [==============================] - 243s 121ms/step - loss: 0.0061 - acc: 0.9686 - val_loss: 0.0071 - val_acc: 0.9914
Epoch 3/100
2000/2000 [==============================] - 242s 121ms/step - loss: 0.0055 - acc: 0.9693 - val_loss: 0.0069 - val_acc: 0.9920
Epoch 4/100
2000/2000 [==============================] - 239s 120ms/step - loss: 0.0053 - acc: 0.9695 - val_loss: 0.0061 - val_acc: 0.9926
Epoch 5/100
2000/2000 [==============================] - 240s 120ms/step - loss: 0.0052 - acc: 0.9696 - val_loss: 0.0063 - val_acc: 0.9925
Epoch 6/100
2000/2000 [==============================] - 241s 120ms/step - loss: 0.0050 - acc: 0.9698 - val_loss: 0.0060 - val_acc: 0.9927
Epoch 7/100
2000/2000 [==============================] - 239s 120ms/step - loss: 0.0049 - acc: 0.9699 - val_loss: 0.0059 - val_acc: 0.9929
Epoch 8/100
2000/2000 [====

### Train residual unet + pix2pix